In [18]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import RadiusNeighborsClassifier
import matplotlib.pyplot as plt
import math
from collections import defaultdict
import operator
import scipy as sp
from sklearn import preprocessing
import json
import operator

In [3]:
data_location_AT = [r"D:\Storage\Research\data\ATER",
                 r"D:\Storage\Research\data\ATERDD",
                 r"D:\Storage\Research\data\ATGEO",
                 r"D:\Storage\Research\data\ATGEOGD",
                 r"D:\Storage\Research\data\ATHGG",
                 r"D:\Storage\Research\data\ATSF",
                 r"D:\Storage\Research\data\ATSFDD",
                 r"D:\Storage\Research\data\ATSticky",
                 r"D:\Storage\Research\data\ATOriginal"]

data_location_CE = [r"D:\Storage\Research\data\CEER",
                 r"D:\Storage\Research\data\CEERDD",
                 r"D:\Storage\Research\data\CEGEO",
                 r"D:\Storage\Research\data\CEGEOGD",
                 r"D:\Storage\Research\data\CEHGG",
                 r"D:\Storage\Research\data\CESF",
                 r"D:\Storage\Research\data\CESFDD",
                 r"D:\Storage\Research\data\CESticky",
                 r"D:\Storage\Research\data\CEOriginal"]

data_location_DM = [r"D:\Storage\Research\data\DMER",
                 r"D:\Storage\Research\data\DMERDD",
                 r"D:\Storage\Research\data\DMGEO",
                 r"D:\Storage\Research\data\DMGEOGD",
                 r"D:\Storage\Research\data\DMHGG",
                 r"D:\Storage\Research\data\DMSF",
                 r"D:\Storage\Research\data\DMSFDD",
                 r"D:\Storage\Research\data\DMSticky",
                 r"D:\Storage\Research\data\DMOriginal"]

data_location_EC = [r"D:\Storage\Research\data\ECER",
                 r"D:\Storage\Research\data\ECERDD",
                 r"D:\Storage\Research\data\ECGEO",
                 r"D:\Storage\Research\data\ECGEOGD",
                 r"D:\Storage\Research\data\ECHGG",
                 r"D:\Storage\Research\data\ECSF",
                 r"D:\Storage\Research\data\ECSFDD",
                 r"D:\Storage\Research\data\ECSticky",
                 r"D:\Storage\Research\data\ECOriginal"]

data_location_HS = [r"D:\Storage\Research\data\HSER",
                 r"D:\Storage\Research\data\HSERDD",
                 r"D:\Storage\Research\data\HSGEO",
                 r"D:\Storage\Research\data\HSGEOGD",
                 r"D:\Storage\Research\data\HSHGG",
                 r"D:\Storage\Research\data\HSSF",
                 r"D:\Storage\Research\data\HSSFDD",
                 r"D:\Storage\Research\data\HSSticky",
                 r"D:\Storage\Research\data\HSOriginal"]

data_location_MM = [r"D:\Storage\Research\data\MMER",
                 r"D:\Storage\Research\data\MMERDD",
                 r"D:\Storage\Research\data\MMGEO",
                 r"D:\Storage\Research\data\MMGEOGD",
                 r"D:\Storage\Research\data\MMHGG",
                 r"D:\Storage\Research\data\MMSF",
                 r"D:\Storage\Research\data\MMSFDD",
                 r"D:\Storage\Research\data\MMSticky",
                 r"D:\Storage\Research\data\MMOriginal"]

data_location_RN = [r"D:\Storage\Research\data\RNER",
                 r"D:\Storage\Research\data\RNERDD",
                 r"D:\Storage\Research\data\RNGEO",
                 r"D:\Storage\Research\data\RNGEOGD",
                 r"D:\Storage\Research\data\RNHGG",
                 r"D:\Storage\Research\data\RNSF",
                 r"D:\Storage\Research\data\RNSFDD",
                 r"D:\Storage\Research\data\RNSticky",
                 r"D:\Storage\Research\data\RNOriginal"]

data_location_SC = [r"D:\Storage\Research\data\SCER",
                 r"D:\Storage\Research\data\SCERDD",
                 r"D:\Storage\Research\data\SCGEO",
                 r"D:\Storage\Research\data\SCGEOGD",
                 r"D:\Storage\Research\data\SCHGG",
                 r"D:\Storage\Research\data\SCSF",
                 r"D:\Storage\Research\data\SCSFDD",
                 r"D:\Storage\Research\data\SCSticky",
                 r"D:\Storage\Research\data\SCOriginal"]

data_location_SP = [r"D:\Storage\Research\data\SPER",
                 r"D:\Storage\Research\data\SPERDD",
                 r"D:\Storage\Research\data\SPGEO",
                 r"D:\Storage\Research\data\SPGEOGD",
                 r"D:\Storage\Research\data\SPHGG",
                 r"D:\Storage\Research\data\SPSF",
                 r"D:\Storage\Research\data\SPSFDD",
                 r"D:\Storage\Research\data\SPSticky",
                 r"D:\Storage\Research\data\SPOriginal"]

In [4]:
def distance(origin, other):
    return np.sum((origin - other) ** 2)**(1/2)

def gamma_mle(data, iterations = 4):
            alphas = np.zeros((1,2)) # each set of data will be given row of alpahs per column of data
            x = np.zeros((1,2))
            #print(np.mean(data))
            #print(np.sum(data == 0))
#             print(data)
            x[:,0] = np.log(np.mean(data))
            x[:,1] = np.mean(np.log(data))
#             print(f"calculated the initial values {x}")
            alphas[:,0] = .5/(x[:,0] - x[:,1])
            
            k = alphas[:,0]
            for i in range(iterations):
#                 print(f'on iteration {i} for calcuating k ({k})')
                digamma = sp.special.digamma(k)
                digamma_prime = sp.special.polygamma(1,k)
                k = 1/(1/k +(x[:,1] - x[:,0] + np.log(k) - digamma)/k**2*(1/k-digamma_prime))
            
            if np.any(np.isnan(k)):
#                 print("for the following data the k became not a number,", data, data.shape)
                print("the data was calculated to have a k of not a number")
            alphas[:, 0] = k
            alphas[:, 1] = np.exp(x[:,0])/alphas[:,0]
            
            return alphas

In [11]:
class FDC():
    
    def __init__(self, x, y, model = 'gamma', threshold = .00):
        self.data = x
        self.labels = y
        self.model = model
        self.threshold = threshold
        self.model = model
        
    def fit(self):

        u_labels = set(self.labels)
        if self.model == 'gamma':
            self.alphas = {label:{} for label in u_labels}

            for label in u_labels:
                fitting_points = self.data[self.labels == label]
                for column in range(self.data.shape[1]):
                    c = gamma_mle(fitting_points[:, column])[0]
                    self.alphas[label][column] = c
                    if self.alphas[label][column][1] == np.nan:
                        self.alphas[label][column] = (sp.stats.mode(fitting_points[:,column])[0], self.alphas[label][column][1])
                    
        elif self.model == 'normal':
            self.alphas = {label:{} for label in u_labels}
            
            for label in u_labels:
                fitting_points = self.data[y == label]
                for column in range(self.data.shape[1]):
                    self.alphas[label][column] = (np.mean(fitting_points[:,column]), np.std(fitting_points[:, column]))
                    if self.alphas[label][column][1] == 0:
                        self.alphas[label][column] = (sp.stats.mode(fitting_points[:,column])[0], self.alphas[label][column][1])
                    # stored as (mean, standard deviation)
            #print(self.alphas)

    def predict(self, point, explicit = False):
        p_scores = {label:[0]*self.data.shape[1] for label in set(self.labels)}
        #print(f"the model being used is {self.model}")
        if self.model == 'gamma':
            for label, f_alphas in self.alphas.items():
                for feature, alpha in f_alphas.items():
                    if np.isnan(alpha[0]):
                        # Need to input a check to see if is just the same as all other numbers and assign there
                        p_scores[label][feature] = 0 if point[feature] != alpha[0] else 1
#                         print("set p score to 1")
                    else:
                        rv = sp.stats.gamma(alpha[0], scale = alpha[1])
                        cdf = rv.cdf(point[feature])
                        p_scores[label][feature] = min(1 - cdf, cdf)
                        # 2 sided
#                 print(f"there are {np.sum(p_scores[label] == 1)} ones in the p_scores")
#                 print(f"the scores are {p_scores[label]}")
#                 print(p_scores)

                p_scores[label] = 1 - np.asarray(p_scores[label])
                p_scores[label] = 1 - np.product(p_scores[label])
                #classify into class with highest lowest probability
            return p_scores if explicit else max(p_scores.items(), key=operator.itemgetter(1))[0]
        
        elif self.model == 'normal':
            for label, f_alphas in self.alphas.items():
                for feature, alpha in f_alphas.items():
                    if alpha[1] != 0:
                        z = (point[feature] - alpha[0])/alpha[1]
                    else:
                        #print(point[feature], alpha[0])
#                         print('normal has a nan')
                        z = 20 if point[feature] != alpha[0] else .5
                    p_scores[label][feature] = np.abs(z)
               
                p_scores[label] = np.max(p_scores[label][p_scores[label] != np.nan])
                # get lowest probability
            # return highest lowest probability
            #print(f"p_scores are {p_scores}")
            return p_scores if explicit else min(p_scores.items(), key=operator.itemgetter(1))[0]
    
    def score(self):
        correct = 0
        total = 0
        for index, point in enumerate(self.data):
#             print(f"predicting index {index}")
            explicit = self.predict(point, explicit = True)
            predict = max(explicit.items(), key=operator.itemgetter(1))[0]
            if predict == self.labels[index]:
                correct += 1
            else:
                print(f"incorrect class prediction {predict} for class {self.labels[index]}: {explicit}")
            total += 1
        
        return correct/total

In [21]:
print('actually running')
num = 8
cats = num if num <= 8 else 8
X = []
y = []
for i in range(cats):
        x = pd.read_csv(data_location_AT[i], header = None, sep = ' ').iloc[:,:].values
        for b in x:
            X.append(b)
            y.append(i)
            
def customNorm(X):
    x = X
    for feature in range(X.shape[1]):
                x[:,feature] = X[:,feature]/np.max(X[:, feature])
    return x            

# #print(len(X), len(X[0]))
X = np.asarray(X)
#print("data shape should be:", X.shape)
y = np.asarray(y) 
print(np.sum(X+.01 == 0))
x = customNorm(X)
print(np.max(x))
print(np.sum(x == 0))
test_class = FDC(x,y, model = 'gamma')
# print("going to fit the class")
test_class.fit()
# for index, point in enumerate(x):
#     print(test_class.predict(point, explicit = True))
# print(test_class.alphas)
test_class.score()

actually running
0
1.0
6570
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k 

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in subtract
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in subtract
  del sys.path[0]
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in log


incorrect class prediction 3 for class 0: {0: 0.32030046140307267, 1: 0.007621717635557412, 2: 1.1102230246251565e-16, 3: 0.6749335048456396, 4: 0.4518066626796139, 5: 0.0, 6: 0.01000827667177595, 7: 0.17311633850561803}
incorrect class prediction 3 for class 0: {0: 0.5694353880886025, 1: 0.0003451421036009261, 2: 0.0007755336429173809, 3: 0.6060802861673136, 4: 0.4859011404088457, 5: 0.0, 6: 0.012353105719556612, 7: 7.271891842131062e-06}
incorrect class prediction 3 for class 0: {0: 0.6098479614459296, 1: 0.14048918529457566, 2: 8.855138844410249e-13, 3: 0.6689917118354982, 4: 0.4605805290758159, 5: 0.0, 6: 0.09517298047817946, 7: 0.3894353175208769}
incorrect class prediction 6 for class 1: {0: 0.49865345672400707, 1: 0.9350250332713061, 2: 9.714271831384735e-09, 3: 8.083148947646279e-05, 4: 0.6157040547911199, 5: 0.16059746399789954, 6: 0.9604468305316733, 7: 0.03778849366546322}
incorrect class prediction 6 for class 1: {0: 0.49865345672400707, 1: 0.9139883502042239, 2: 9.71427183

incorrect class prediction 6 for class 1: {0: 0.49865345672400707, 1: 0.9499866095423991, 2: 9.714271831384735e-09, 3: 6.670214151571763e-05, 4: 0.5996206900677563, 5: 0.12341827632283009, 6: 0.9609815378735468, 7: 0.03776619002187165}
incorrect class prediction 6 for class 1: {0: 0.49865345672400707, 1: 0.9309185684231776, 2: 9.714271831384735e-09, 3: 3.5257563820834115e-05, 4: 0.628287177792749, 5: 0.19107520109169007, 6: 0.9623910714121988, 7: 0.041169871756579934}
incorrect class prediction 3 for class 2: {0: 0.0002661596899790064, 1: 0.0015073208549718498, 2: 0.9489382796522858, 3: 0.9562600024438374, 4: 0.6196684452126353, 5: 0.5117256871187731, 6: 0.8006297253273316, 7: 5.840368655363193e-08}
incorrect class prediction 3 for class 2: {0: 3.7584380052635424e-12, 1: 0.0018459663362554446, 2: 0.8712841656530235, 3: 0.977098480071577, 4: 0.7369206222725346, 5: 0.4010143732034597, 6: 0.7847161502986829, 7: 3.1972682029701716e-06}
incorrect class prediction 3 for class 2: {0: 4.107825

incorrect class prediction 3 for class 2: {0: 0.0005640623543186063, 1: 0.0015888547226572403, 2: 0.7350204410114701, 3: 0.9561120529016839, 4: 0.5582202578128391, 5: 0.5575871457424264, 6: 0.7951631889440893, 7: 1.8299932269538743e-07}
incorrect class prediction 3 for class 2: {0: 0.0021025899340541, 1: 0.0015592851551766707, 2: 0.6262674592690516, 3: 0.953549207108141, 4: 0.6361161479943844, 5: 0.5362353980975236, 6: 0.8000108351454869, 7: 1.3828054628017838e-06}
incorrect class prediction 3 for class 2: {0: 5.440092820663267e-15, 1: 0.0018794744679050401, 2: 0.8727917681359341, 3: 0.9781731218541804, 4: 0.7094231276416164, 5: 0.4057417538622742, 6: 0.7767127209515203, 7: 1.5146057988357775e-05}
incorrect class prediction 3 for class 2: {0: 4.508330701658014e-05, 1: 0.0015711040820218791, 2: 0.9275923974452421, 3: 0.9592081305551129, 4: 0.6440867677430611, 5: 0.4957213908151267, 6: 0.7929300839592797, 7: 4.729405533865361e-09}
incorrect class prediction 3 for class 2: {0: 2.055022818

incorrect class prediction 6 for class 3: {0: 0.49919877812696134, 1: 0.5066772992966673, 2: 1.001315752535703e-08, 3: 0.06923962756226898, 4: 0.22070645465567662, 5: 0.4639881169606681, 6: 0.9335088968751007, 7: 0.11056148985976266}
incorrect class prediction 6 for class 3: {0: 0.5667920079780087, 1: 0.3434071538396509, 2: 1.001315752535703e-08, 3: 0.8431377919805483, 4: 0.0895424559930128, 5: 0.06175724055734255, 6: 0.9520517458305654, 7: 0.042305943730973294}
incorrect class prediction 6 for class 3: {0: 0.5104333228011837, 1: 0.40606477756339077, 2: 1.001315752535703e-08, 3: 0.3943828599603483, 4: 0.1734172312288651, 5: 0.03176586327065056, 6: 0.946522048569266, 7: 0.03731476787009491}
incorrect class prediction 6 for class 3: {0: 0.510914862134869, 1: 0.6577018205482081, 2: 1.001315752535703e-08, 3: 0.006994267474424221, 4: 0.3084853994446345, 5: 0.20589176307456158, 6: 0.9182750034946773, 7: 0.30420513776993785}
incorrect class prediction 6 for class 3: {0: 0.4991987781269649, 1:

incorrect class prediction 6 for class 5: {0: 0.0, 1: 0.1950312383834356, 2: 0.0003446784083818155, 3: 0.8892948395232393, 4: 0.2296094400506341, 5: 0.9342644676054774, 6: 0.9522085691388451, 7: 0.0}
incorrect class prediction 6 for class 5: {0: 0.0, 1: 0.08857267180294837, 2: 0.0, 3: 0.8332390592499039, 4: 0.21808273412401402, 5: 0.6941835986100742, 6: 0.9545963245930866, 7: 0.0}
incorrect class prediction 6 for class 5: {0: 0.0, 1: 0.31559432973577894, 2: 8.093525849517391e-14, 3: 0.8537935358105654, 4: 0.2054490030933842, 5: 0.852197842236117, 6: 0.9544847554572268, 7: 0.0}
incorrect class prediction 6 for class 5: {0: 0.0, 1: 0.00040008012749293, 2: 0.32769925829223767, 3: 0.9366135020109043, 4: 0.5705976570839195, 5: 0.7365743658247532, 6: 0.9531799179275676, 7: 0.0}
incorrect class prediction 6 for class 5: {0: 0.0, 1: 0.24473133816302872, 2: 2.1094237467877974e-15, 3: 0.8610615143591847, 4: 0.21798119691903106, 5: 0.8907563446679959, 6: 0.9643953686933179, 7: 0.0}
incorrect clas

incorrect class prediction 3 for class 6: {0: 0.033702330934288316, 1: 0.6082431791725302, 2: 0.00045388432919124533, 3: 0.8727147729346011, 4: 0.17630415926758014, 5: 0.0006968702178531405, 6: 0.7410589641185623, 7: 0.46944346962237415}
incorrect class prediction 1 for class 7: {0: 0.00020765169806613581, 1: 0.6595225541613852, 2: 7.940650581517161e-09, 3: 0.6393210126724165, 4: 0.1937401918896534, 5: 0.0, 6: 0.2578114670256416, 7: 0.35555383402767426}
incorrect class prediction 1 for class 7: {0: 1.56861070941261e-05, 1: 0.6714920061782955, 2: 5.950460124637402e-09, 3: 0.6362060888402727, 4: 0.19722467910145813, 5: 0.0, 6: 0.2569224506294969, 7: 0.5432612494033031}
incorrect class prediction 1 for class 7: {0: 0.006470377944210015, 1: 0.6621171494742855, 2: 3.783230640985735e-07, 3: 0.6440040125116836, 4: 0.20335464609079734, 5: 0.0, 6: 0.26196460375559893, 7: 0.632898428456875}
incorrect class prediction 1 for class 7: {0: 0.00024464384903721825, 1: 0.6590845393055758, 2: 5.94843158

0.9399519615692554

In [23]:
print(test_class.alphas)

# print('apples')
# test_class.fit()
# print(test_class.alphas)

{0: {0: array([3.54253728e+04, 1.68147324e-05]), 1: array([8.66735242e+03, 4.27283689e-07]), 2: array([6.91501676e+01, 2.39494599e-06]), 3: array([3.80211700e+03, 1.32890361e-06]), 4: array([3.51316673e+03, 1.14233481e-09]), 5: array([3.19158066e+02, 3.38313885e-07]), 6: array([6.37869531e+01, 3.64767413e-08]), 7: array([nan, nan]), 8: array([nan, nan]), 9: array([2.11501983e+03, 2.38868752e-06]), 10: array([2.00578353e+03, 1.03223566e-08]), 11: array([1.66739377e+03, 2.09561588e-12]), 12: array([5.46967609e+01, 2.72992275e-09]), 13: array([6.22625121e+01, 8.14214222e-08]), 14: array([5.43996610e+01, 5.89672096e-11]), 15: array([7.58038540e+02, 2.60170839e-07]), 16: array([2.85907943e+02, 4.94932840e-08]), 17: array([nan, nan]), 18: array([nan, nan]), 19: array([nan, nan]), 20: array([nan, nan]), 21: array([6.48477718e+00, 4.66509326e-08]), 22: array([nan, nan]), 23: array([nan, nan]), 24: array([nan, nan]), 25: array([nan, nan]), 26: array([nan, nan]), 27: array([nan, nan]), 28: array

In [43]:
b = test_class.fitting_points
# gamma_mle(test_class.fitting_points)

In [44]:
b.shape

(499, 30)

In [50]:
print(gamma_mle(b))

# for i in range(0,30):
#     print(gamma_mle(b[i:i+1]))

calculated the initial values [[-0.52740254 -0.52741631]
 [-3.31946844 -3.31959069]
 [-2.43447938 -2.43472225]
 [-1.37889294 -1.3790166 ]
 [-6.10955308 -6.1107118 ]
 [-2.12049642 -2.12081576]
 [-3.39901346 -3.39937438]
 [-4.12181373 -4.12235697]
 [-4.67737363 -4.67874453]
 [-0.21166021 -0.21184648]
 [-3.42632068 -3.42669689]
 [-8.46478352 -8.46847948]
 [-3.44635458 -3.44673115]
 [-1.85492669 -1.85520149]
 [-5.6846261  -5.68615858]
 [-0.64773393 -0.64819516]
 [-1.32724498 -1.32759774]
 [-5.72465105 -5.72548023]
 [-3.03703626 -3.03805863]
 [-2.81474407 -2.8152867 ]
 [-4.40866245 -4.40972183]
 [-0.62973091 -0.63023334]
 [-8.15713281 -8.15858293]
 [-4.64030746 -4.64142124]
 [-3.30267473 -3.30341271]
 [-2.46909721 -2.47011191]
 [-4.84420107 -4.84543788]
 [-3.90380056 -3.90508427]
 [-5.16169385 -5.16363387]
 [-7.00163026 -7.00571636]]
on iteration 0 for calcuating k ([36307.42795691  4090.08648655  2058.64776525  4043.45231072
   431.51338682  1565.75090197  1385.35067712   920.41414203
   3

In [ ]:
test_class = FDC(x,y, model = 'gamma')
# print("going to fit the class")
test_class.fit()
# for index, point in enumerate(x):
#     print(test_class.predict(point, explicit = True))
# print(test_class.alphas)
test_class.score()

In [12]:
print('actually running')
num = 8
cats = num if num <= 8 else 8
X = []
y = []
for i in range(cats):
        x = pd.read_csv(data_location_AT[i], header = None, sep = ' ').iloc[:,:].values
        for b in x:
            X.append(b)
            y.append(i)
            
def customNorm(X):
    x = X
    for feature in range(X.shape[1]):
                x[:,feature] = X[:,feature]/np.max(X[:, feature])
    return x            

# #print(len(X), len(X[0]))
X = np.asarray(X)
#print("data shape should be:", X.shape)
y = np.asarray(y) 
x = customNorm(X)

X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.33, random_state=42)

print(X_train.shape, y_train.shape)
test_class = FDC(X_train,y_train, model = 'gamma')

test_class.fit()

actually running
(2510, 30) (2510,)
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to have a k of not a number
the data was calculated to h

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in subtract
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in subtract
  del sys.path[0]
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in log


In [15]:
correct = 0
count = 0
for index, point in enumerate(X_test):
    if test_class.predict(point) == y_test[index]:
        correct += 1
    count += 1
#     print(index)
print(f"correct percentage: {correct/count}")

correct percentage: 0.9434114793856103


In [16]:
iris = r"D:\Tim Huang\Documents\knnNullClassifiers-master\Nearest-Neightbor-Classification\data\Iris Data\iris.txt"
X = pd.read_csv(iris, header = None, sep = '   ').iloc[:,:].values
y = X[:,-1].astype(int)
X = X[:,: -2]

iris_class = FDC(X, y)
iris_class.fit()
iris_class.score()

incorrect class prediction 2 for class 0: {0: 0.39265946615278646, 1: 0.2782665421812992, 2: 0.4769500911857869}
incorrect class prediction 2 for class 0: {0: 0.1496659266489735, 1: 0.2597193507312102, 2: 0.49952472065766673}

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  



incorrect class prediction 1 for class 0: {0: 0.1596496723771671, 1: 0.4213243176257634, 2: 0.10328794086527215}
incorrect class prediction 1 for class 0: {0: 0.2485361330885063, 1: 0.30884666733932675, 2: 0.06899613690924689}
incorrect class prediction 2 for class 0: {0: 0.30065206215422036, 1: 0.22118454130623666, 2: 0.44932694006843654}
incorrect class prediction 2 for class 1: {0: 0.3013135398897584, 1: 0.2471425564000116, 2: 0.4565999808043074}
incorrect class prediction 2 for class 1: {0: 0.2761443952216186, 1: 0.4621944428172827, 2: 0.5584509633006409}
incorrect class prediction 2 for class 1: {0: 0.2093954240543341, 1: 0.2601552544652328, 2: 0.6063538669599287}
incorrect class prediction 2 for class 1: {0: 0.05160658030991028, 1: 0.6311496360370034, 2: 0.6318997059082432}
incorrect class prediction 2 for class 1: {0: 0.3943547426224403, 1: 0.3796330222516776, 2: 0.47800489803421364}
incorrect class prediction 0 for class 1: {0: 0.3729902870043207, 1: 0.13598930158180156, 2: 0.

0.7702702702702703

In [36]:
neigh = KNeighborsClassifier(n_neighbors=342)
neigh.fit(X_train, y_train)
neigh.score(X_test,y_test)

0.9417946645109135